In [1]:
import numpy as np

In [2]:
from sklearn.metrics import make_scorer

In [3]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import category_encoders

In [5]:
import xgboost as xgb

In [6]:
from sklearn.model_selection import GridSearchCV

In [7]:
import pandas as pd

In [8]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [9]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [38]:
processed = pd.read_csv('features_01.csv')

In [39]:
processed = processed.set_index('person')
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,visitas_por_dia
person,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,14,6.0,Referral,2.000000
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,0,16.0,Direct,1.214286
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,66,10.0,Paid,1.000000
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,5,20.6,Direct,0.600000
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,2,17.0,Paid,1.000000


In [41]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [42]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

/home/javier/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [43]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,visitas_por_dia
person,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,6,1,14,6.000000,Referral,2.000000
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,17,1,2,17.000000,Paid,1.000000
001802e4,ad campaign hit,iPhone 6S,Smartphone,0,1,4,19,1,0,19.000000,Paid,1.000000
0019e639,viewed product,Motorola Moto G2 3G Dual,Computer,2,19,189,471,115,9,4.095652,Paid,0.165217
001b0bf9,checkout,Samsung Galaxy S8 Plus,Ninguno,1,0,2,7,1,9,7.000000,Ninguno,0.000000


In [44]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [45]:
encoded_training.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.007335,0.042654,0.069914,0,2,0,6,1,14,6.000000,0.065251,2.000000
1,0.053592,0.049793,0.034791,0,1,3,17,1,2,17.000000,0.042291,1.000000
2,0.018508,0.034037,0.034791,0,1,4,19,1,0,19.000000,0.042291,1.000000
3,0.055779,0.026954,0.069914,2,19,189,471,115,9,4.095652,0.042291,0.165217
4,0.007335,0.054054,0.017361,1,0,2,7,1,9,7.000000,0.017361,0.000000


In [46]:
params={
    'n_estimators':[1,2,3,4,5,6,7,8,9,10]
}
grid = GridSearchCV(xgb.XGBRegressor(),params,scoring=make_scorer(roc_auc_score),cv=2,verbose=0)

In [47]:
xtr,xte,ytr,yte=train_test_split(encoded_training,training_labels)

In [48]:
grid.fit(xtr,ytr)

GridSearchCV(cv=2, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(roc_auc_score), verbose=0)

In [49]:
grid.score(X=xte,y=yte)

0.7628988082019396

In [50]:
xgbc= grid.best_estimator_

In [51]:
xgbc.fit(xtr,ytr)
p=xgbc.predict(xte)
roc_auc_score(y_score=p,y_true=yte)

0.7628988082019396

In [52]:
xgbc.fit(encoded_training,training_labels)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [53]:
prediction_submit = xgbc.predict(encoded_predict)

In [54]:
predict_features['label'] = prediction_submit

In [55]:
predict_features['label'].value_counts()

0.186529    4887
0.191011    3290
0.203393    1272
0.207874     830
0.208100     546
0.199288     320
0.236024     268
0.217443     224
0.195618     224
0.204273     219
0.229057     217
0.191137     212
0.212581     165
0.194807     156
0.188635     146
0.205390     142
0.200909     127
0.216152     126
0.187311     113
0.216377     111
0.231162     107
0.206798     106
0.204805     106
0.225721     106
0.233179      97
0.232198      96
0.205045      95
0.191792      85
0.302150      80
0.238937      77
            ... 
0.430509       1
0.228858       1
0.254590       1
0.207465       1
0.246551       1
0.399437       1
0.244406       1
0.270044       1
0.285656       1
0.383923       1
0.231090       1
0.508632       1
0.228720       1
0.301182       1
0.477900       1
0.231148       1
0.274862       1
0.394887       1
0.254257       1
0.363592       1
0.254172       1
0.274969       1
0.379060       1
0.212938       1
0.493832       1
0.285179       1
0.410166       1
0.347616      

In [56]:
subm = predict.merge(predict_features['label'].reset_index(),how='left',on='person')

In [57]:
subm = subm.set_index('person')

In [58]:
subm.to_csv('submit.csv')